# 01. ML 모델링 복습


#### 머신러닝 코드 구조 <br>

![이미지](https://github.com/DA4BAM/dataset/blob/master/new_code.png?raw=true "code step1")



## 00 환경준비

### 01.Import

In [ ]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 02.read_csv

In [ ]:
# 데이터를 불러옵시다.
titanic = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/titanic.0.csv')

## 10.데이터 이해

![이미지](https://github.com/DA4BAM/dataset/blob/master/new_code.png?raw=true "code step1")

### 11.둘러보기

In [ ]:
# 상/하위 몇개 행을 살펴 봅시다.
titanic.head()

In [ ]:
# 각 칼럼의 타입을 살펴 봅시다.
titanic.dtypes

In [ ]:
# 행, 열의 개수를 살펴 봅시다.
titanic.shape

### 12.기초통계량

In [ ]:
# 숫자형 변수들의 통계량을 살펴 봅니다.
titanic.describe()

In [ ]:
# 범주형 변수의 통계량을 살펴 봅니다.
titanic['Survived'].value_counts()

In [ ]:
titanic['Survived'].value_counts() / titanic.shape[0]

In [ ]:
# 다른 범주형 변수(Pclass, Sex, Embarked) 에 대해서도 통계량를 살펴봅시다.



In [ ]:
# 각 변수별 NaN 현황 파악하기
titanic.isnull().sum()

### 13.탐색하기

#### 1단계 : 개별변수의 분포

In [ ]:
# Target 변수의 분포
titanic['Survived'].value_counts().plot.bar()
plt.show()

In [ ]:
# matplotlib histogram
plt.hist(titanic['Age'], color = 'green', edgecolor = 'black',bins = 20)
plt.show()

In [ ]:
plt.hist(titanic['Fare'], color = 'green', edgecolor = 'black',bins = 50)
plt.show()

#### 2단계 : x -> y

In [ ]:
g = sns.FacetGrid(data=titanic, hue='Survived', size = 5)
g.map(sns.distplot, 'Age', kde=True, hist=False)
plt.show()

In [ ]:
g = sns.FacetGrid(data=titanic, hue='Survived', size = 5)
g.map(sns.distplot, 'Fare', kde=True, hist=False)
plt.show()

In [ ]:
titanic.groupby(['Sex', 'Survived']).size().unstack().plot.bar(stacked=True)
plt.show()

In [ ]:
df_bar = (titanic.groupby(['Sex','Survived'])['PassengerId'].count()/titanic.groupby(['Sex'])['PassengerId'].count())
df_bar.unstack().plot.bar(stacked=True)
plt.show()

In [ ]:
df_bar = (titanic.groupby(['Pclass','Survived'])['PassengerId'].count()/titanic.groupby(['Pclass'])['PassengerId'].count())
df_bar.unstack().plot.bar(stacked=True)
plt.show()

## 20.데이터 준비

![이미지](https://github.com/DA4BAM/dataset/blob/master/new_code.png?raw=true "code step1")

### 21.변수 정리

In [ ]:
titanic.head()

In [ ]:
drop_vars = ['PassengerId','Name','Ticket','Cabin']
titanic0 = titanic.drop(drop_vars, axis=1)
titanic0.head()

### 22.NA 처리

In [ ]:
titanic0.isnull().sum()

### 23.Feature Engineering

### 24.Dummy Variable


익혀야 할 방법들
pd.get_dummies, pd.concat, (Pandas Dataframe).drop

In [ ]:
titanic0.columns

In [ ]:
# 범주형(명목형) 변수를 가변수로 만들고 붙여봅시다.
dummy_vars = ['Pclass', 'Sex','Embarked']
for each in dummy_vars:
    dummies = pd.get_dummies(titanic0[each], prefix=each, drop_first=True)
    titanic0 = pd.concat([titanic0, dummies], axis=1)

titanic0.head()

In [ ]:
# 불필요한 변수들 제거 : 가변수화 하기 전 변수, id, 일련번호 등.
titanic1 = titanic0.drop(dummy_vars, axis=1)
titanic1.head()

### 25.Data Split

sklearn의 데이터 분할 함수를 사용해 봅시다.

* 요인, x, feature, 조작변수, 통제변수, 리스크백터, Input, (독립변수)
* 결과, y, target, label, Output, (종속변수)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# features와 target 분리
X = titanic1.drop('Survived', axis=1)
y = titanic1.iloc[:, 0]

* 1단계 - train_val : test로 나누기

In [ ]:
# 전체에서 train_val : test = 8 : 2
train_val_x, test_x, train_val_y, test_y = train_test_split(X, y, test_size=0.2)

* 2단계 - train : val 로 나누기

In [ ]:
# train_val에서 train : val = 8 : 2
train_x, val_x, train_y, val_y = train_test_split(train_val_x, train_val_y, test_size=0.2)

In [ ]:
print(train_x.shape)
print(val_x.shape)
print(test_x.shape)

### 26.Scaling & NA처리




* NA 처리 : knn imputation

In [ ]:
# 필요한 함수 불러오기
from sklearn.impute import KNNImputer
# 선언한다.
imputer = KNNImputer()

# 적용한다.
train_x = imputer.fit_transform(train_x)
val_x = imputer.transform(val_x)
test_x = imputer.transform(test_x)

* scaling : min-max 방식

In [ ]:
# 필요한 함수 로딩
from sklearn.preprocessing import MinMaxScaler

# 함수 선언
scaler = MinMaxScaler()

# 함수 적용하기
train_x = scaler.fit_transform(train_x)
val_x = scaler.transform(val_x)
test_x = scaler.transform(test_x)

### 27.Dataframe to Numpy array

In [ ]:
type(train_x)

## 30.모델링

![이미지](https://github.com/DA4BAM/dataset/blob/master/new_code.png?raw=true "code step1")

**빈 칸을 채우며 모델링을 수행해 봅시다.**

### 31.import

In [ ]:
# Logistic Regression Model
from sklearn.linear_model import LogisticRegression        

### 32.모델선언

In [ ]:
# 모델 선언
model = LogisticRegression()

### 33.모델링(학습)

In [ ]:
model.fit(train_x, train_y)

### 34.예측

In [ ]:
# validation 으로 예측값을 뽑자.
val_pred = model.predict(val_x)

### 35.평가

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# confusion matrix
print(confusion_matrix(val_y, val_pred))

In [ ]:
# classification report
print(classification_report(val_y, val_pred))

In [ ]:
print(accuracy_score(val_y, val_pred))